In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('./data/fl_voter_reg/fl_reg_name_race.csv', nrows=SAMPLE)
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian']) == False].copy()

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,9263
hispanic,26516
nh_black,20834
nh_white,117081


## Preprocessing the input data

In [2]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1146
Max feature len = 27, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [3]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

/opt/venv/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


790244 train sequences
197562 test sequences
Pad sequences (samples x time)
X_train shape: (790244, 20)
X_test shape: (197562, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (790244, 4)
y_test shape: (197562, 4)


In [4]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            36672     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 119,620
Trainable params: 119,620
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 711219 samples, validate on 79025 samples
Epoch 1/15
 - 784s - loss: 0.6189 - acc: 0.7796 - val_loss: 0.5915 - val_acc: 0.7872
Epoch 2/15
 - 629s - loss: 0.5826 - acc: 0.7902 - val_loss: 0.5719 - val_acc: 0.7934
Epoch 3/15
 - 452s - loss: 0.5671 - acc: 0.7953 - val_loss: 0.5611 - val_acc: 0.7973
Epoch 4/15
 - 454s - loss: 0.5585 - acc: 0.7976 - val_loss: 0.5534 - val_acc: 0.7994
Epoch 5/15
 - 463s - loss: 0.5524 - acc: 0.7994 - val_loss: 0.5492 - val_acc: 0.8002
Epoch 6/15
 - 761s - loss: 0.5489 - acc: 0.8001 - val_loss: 0.5471 - val_acc: 0.8009
Epoch 7/15
 - 780s - loss: 0.5462 - acc: 0.8014 - val_loss: 0.5444 - val_acc: 0.8022
Epoch 8/15
 - 788s - loss: 0.5440 - acc: 0.8017 - val_loss: 0.5432 - val_acc: 0.8033
Epoch 9/15
 - 671s - loss: 0.5423 - acc: 0.8024 - val_loss: 0.5421 - val_acc: 0.8040
Epoch 10/15
 - 473s - loss: 0.5412 - acc: 0.8026 - val_loss: 0.5411 - val_acc: 0.8043
Epoch 11/15
 - 468s - loss: 0.5401 - acc: 0.8033 - val_loss: 0.5395 - val_acc: 0.8047
Epo

In [6]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.5351445410388976
Test accuracy: 0.8069314949200957


## Confusion Matrix

In [7]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

      asian       0.77      0.41      0.54      4527
   hispanic       0.74      0.70      0.72     18440
   nh_black       0.64      0.21      0.32     28586
   nh_white       0.82      0.95      0.88    146009

avg / total       0.79      0.81      0.78    197562

[[  1867    307    103   2250]
 [    78  12832    154   5376]
 [   121    358   5972  22135]
 [   358   3870   3033 138748]]


## Save model

In [8]:
model.save('./models/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [9]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')